# POC Unique_ID

 Atual notebook com as funçoes para processamento de PDF Pesquisavel e Raster e a criaçao dos Dataframes de forma dependente e unica

### Modules & config

In [1]:
import os
import sys
import shutil
import platform
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
import PyPDF2




import csv
import json
import pandas as pd


import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import pytesseract

# Modulos da solucao
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron


'pt_BR.utf8'

In [ ]:
#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


####Config Processamento Pipeline

# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 7. Path para DFs e CSVs exportados
export_path = "pipeline_extracao_documentos/6_geral_administacao/exports"

# 8. Path para lixeira
root_garbage_path = "pipeline_extracao_documentos/0_lixeira"



# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

#Modelo atual
#tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

# definindo localizadcao para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")

## 1. POC Unique_ID

### P1 - Funcoes para e-mail e extracao

In [2]:
# 1. Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash

# 2. Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())

# 3. XXX Busca proximo Batch
def busca_proximo_batch():
    # Abre o arquivo Excel e lê a coluna 'batch'
    df = pd.read_excel("pipeline_extracao_documentos/6_geral_administacao/exports/df_documento_recebido.xlsx", usecols=["batch"])
    # Pega o último valor da coluna 'batch'
    last_value = df.iloc[-1, 0]
    
    # Extraí o número do último batch e adiciona 1 para o próximo
    last_number = int(last_value.split("_")[1])
    next_number = last_number + 1
    
    # Forma o nome do próximo batch
    next_batch = f"Batch_{next_number}"
    
    return next_batch    

# 4. Função para verificar e criar a pasta se não existir
def check_and_create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
# 5. converte nome do arquivo
def conv_filename(title):
    
    # Divide o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remove acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substiti espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remove quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converte para minúsculas
    filename = filename.lower()

    # Adiciona a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 6. converte nome do arquivo retirando extensao
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename         

# 7. funçao que MOVE documentos e gera add_log_transaction_entry para df_log_transctions
def move_doc_processed_file(batch_name, src_path, tgt_path):
    
    function = "move_doc_processed_file"
    source_path = src_path
    file = os.path.basename(source_path)
    sub_dir = os.path.join(tgt_path, batch_name)
    destination_path = os.path.join(sub_dir, file)
    document_action = "move_processed_file"
    transaction_detail = (f'document {file} moved by: {function}')
    df_move = pd.DataFrame()
    try:
        document_unique_id = get_document_id_by_file(batch_name, file)
        check_and_create_folder(destination_path)
        shutil.move(source_path, destination_path)
        sucess = True
        move_log = add_log_transaction_entry(document_unique_id, batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess)
    except Exception as e:
        print(f"Erro ao mover documento: {e}")
        sucess = False
    
    return move_log    

# 8. Função para adicionar um novo registro em df_source
def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data

# 9. Add nova linha para atualizar df_log_transctions
def add_log_transaction_entry(document_unique_id,batch_name, file, document_action, src_path, tgt_path, transaction_detail, sucess=True):

    data_log = {
        'Dt_Time': cron.timenow_pt_BR(),
        'Batch': batch_name,
        'File' : file,
        'Unique_ID': document_unique_id,
        'Action': document_action,
        'Sorce': src_path,
        'Target': tgt_path,
        'Transction_Detail': transaction_detail,
        'Sucess': sucess,    
    }
    
        
    return data_log


# 10. Consulta multiparametros
def filtrar_df(df, **kwargs):
    query = " & ".join(f"{key} == @kwargs['{key}']" for key in kwargs)
    result = df.query(query)
    return result

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id


# 12. Busca filhos - simples
def get_children(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Parent_Unique_ID=document_unique_id)


# 13. Busca pai -simples
def get_father(batch, file_path):
    
    file = os.path.basename(file_path)
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    parent_unique_id = result['Parent_Unique_ID'].values[0]
    # Buscando todos dos filhos de um documento
    return filtrar_df(df_id_relations, Batch=batch, Unique_ID=parent_unique_id)


# 14. Pesquiso pai pelo Unique_ID e trago um dict
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }


# 15. Pesquiso pai pelo Unique_ID (document_parent_unique_id) e cria DICT
def get_father_by_unique_id(batch, document_parent_unique_id):
    
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=document_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }
    
# 16. Pesquiso pai pelo file do filho e cria DICT
def get_father_data_by_children_file(batch, file):
    
    src_result = filtrar_df(df_id_relations, Batch=batch, File=file)
    src_parent_unique_id = src_result['Parent_Unique_ID'].values[0]
    result = filtrar_df(df_id_relations, Batch=batch, Unique_ID=src_parent_unique_id)
    document_batch = result['Batch'].values[0]
    document_data = result['Data'].values[0]
    document_file = result['File'].values[0]
    document_type = result['Type'].values[0]
    document_level = result['Level'].values[0]
    document_unique_id = result['Unique_ID'].values[0]
    document_parent_unique_id = result['Parent_Unique_ID'].values[0]
    document_hash = result['Hash'].values[0]
    document_file_path = result['File_Path'].values[0]
    
    return {
        'Batch': document_batch, 
        'Data': document_data,
        'File' : document_file,
        'Type': document_type,
        'Level': document_level,
        'Unique_ID': document_unique_id,
        'Parent_Unique_ID': document_parent_unique_id,
        'Hash': document_hash,
        'File_Path': document_file_path,
    }        
        

# 17. Busca o 'Unique_ID' para definir o Parent_Unique_ID sem considerar 'Level'
def get_parent_unique_id(df_id_relations, batch_name, file, type):
    try:
        parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type)]['Unique_ID'].values[0]
    except IndexError:
        parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return parent_unique_id


# 18. funcao para trazer somente o 'Unique_ID'
def get_document_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Unique_ID'].values[0]
    except IndexError:
        document_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_unique_id


# 19. funcao para trazer somente o 'Parent_Unique_ID'
def get_document_parent_unique_id(df_id_relations, batch_name, file, type, level):
    try:
        document_parent_unique_id = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)]['Parent_Unique_ID'].values[0]
    except IndexError:
        document_parent_unique_id = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_parent_unique_id


# 20. funçao para trazer toda a row de df_id_relations para o documento
def get_document_id_relations(df_id_relations, batch_name, file, type, level):
    try:
        document_id_relations = df_id_relations[(df_id_relations['Batch'] == batch_name) & (df_id_relations['File'] == file) & (df_id_relations['Type'] == type) & (df_id_relations['Level'] == level)].values[0]
    except IndexError:
        document_id_relations = None
        print(f"Unique_ID para Batch {batch_name} e type: {type} nao encontrado em df_id_relations.")
    return document_id_relations




# EXEMPLOS de Pesquisa DFss
    # get_document_unique_id(df_id_relations, batch_name, file, type, level)

    # # Busca somente o 'Parent_Unique_ID'
    # get_document_parent_unique_id(df_id_relations, batch_name, file, type, level)


    # #Busca todos os dados da row do documento encontrado
    # document_id_relations = get_document_id_relations(df_id_relations, batch_name, file, type, level)

    # document_batch = document_id_relations[0]
    # document_date = document_id_relations[1]
    # document_name = document_id_relations[2]
    # document_type = document_id_relations[3]
    # document_level = document_id_relations[4]
    # document_unique_id = document_id_relations[5]
    # document_parent_unique_id = document_id_relations[6]
    # document_hash = document_id_relations[7]
    # document_path = document_id_relations[8]

    # # Insercao de um registro pela func add_source_entry
    # file_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/12_attachments/SPA 15082023.rar"

    # file = os.path.basename(file_path)

    # type = "compressed_file_attachment"

    # level = 1

    # parent_unique_id = ''

    # # Adicionando um novo registro (substitua 'batch_name' e 'email' conforme necessário)
    # new_entry = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)

    # df_id_relations = df_id_relations.append(new_entry, ignore_index=True)

    # df_id_relations


# Busca proximo Batch caso nao esteja rodando email
batch_name = busca_proximo_batch()

# 1. Criaçao do DataFrame para armazenar as relações de Unique_ID e Parent_Unique_ID
df_id_relations = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'Parent_Unique_ID', 'Hash', 'File_Path'])

# 2. Criaçao do DataFrame para df_start_pipe:
#df_start_pipe = pd.DataFrame(columns=['Batch', 'Data' ,'File', 'Type', 'Level', 'Unique_ID', 'dt_hora', 'de', 'assunto', 'email', 'Hash'])

df_id_relations

,Batch,Data,File,Type,Level,Unique_ID,Parent_Unique_ID,Hash,File_Path


### Executar a demanda

In [3]:
# 1A - XXX Função para processar o e-mail individualmente: df_mail
def process_start_input(df_id_relations, msg, msg_type, batch_name, file_path, first_doc_file):
    
    file = first_doc_file
    type = msg_type
    if type != "email":
        msg_de = msg['de']
        msg_assunto = msg['assunto']
        msg_email = msg['email']
        msg_type = msg['msg_type']
        orig_dt_time = msg['dt_hora']
        print(f'\norig_dt_time: {orig_dt_time}')
        msg_date_time = cron.convert_email_date(orig_dt_time)
    else:
        msg_de = msg.sender.split('<')[0].strip()
        msg_assunto = msg.subject
        msg_email = msg.sender.split('<')[-1].strip('<>')
        msg_type = type
        original_date_str = msg.date
        msg_date_time = cron.convert_email_date(original_date_str)
        #print(f'original_date_str: {original_date_str} | msg_date_time: {msg_date_time}')
        
    level = 1
    parent_unique_id = ''
    new_entry = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)
    df = pd.DataFrame()
    df_trans_relation = pd.DataFrame()
    df = df.append(new_entry, ignore_index=True)
    df_trans_relation = df
    #print(df_trans_relation)
    
    # Busca valores processados de new_entry
    unique_id = new_entry['Unique_ID']
    file_hash = new_entry['Hash']
    data_process = new_entry['Data']
    
    return {
        'Batch': batch_name,
        'Data' : data_process,
        'File': first_doc_file,
        'Type': type,
        'Level': level,
        'Unique_Id': unique_id,
        'dt_hora': msg_date_time,
        'de': msg_de,
        'assunto': msg_assunto,
        'email': msg_email,
        'Hash': file_hash
    }, new_entry, df_trans_relation 

# 1B - Função principal para processar e-mails
def start_pipe_email(df_id_relations, msg_dir_path, msg_attachment_zip):
    locale.setlocale(locale.LC_TIME, "pt_BR.utf8")
    rows_list = []
    
    check_and_create_folder(msg_attachment_zip)
    
    for root, dirs, files in os.walk(msg_dir_path):
        doc_files = [doc for doc in files if doc.lower().endswith(".msg")]
        if doc_files:
            first_doc_file = doc_files[0]
            file_path = os.path.join(root, first_doc_file)
            msg_type = "email"
            msg = extract_msg.Message(file_path)
            new_row, new_entry, df_trans_relation = process_start_input(df_id_relations, msg, msg_type, batch_name, file_path, first_doc_file)
            rows_list.append(new_row)
            # try:

                
            # except Exception as e:
            #     print(f"Erro ao ler email: {e}")
    
    
    df_start_pipe_email = pd.DataFrame(rows_list)
    
    pasta_destino = msg_attachment_zip
    
    check_and_create_folder(pasta_destino)
    
    with open(file_path) as msg_file:
        msg = Message(msg_file)
    
    total_attch = len(msg.attachments)
    
    i = 0
    # Loop para salvar cada anexo
    for i in range(total_attch):
        attachment = msg.attachments[i]
        caminho_completo_anexo = os.path.join(pasta_destino, attachment.filename)

        #print(caminho_completo_anexo)
        with attachment.open() as attachment_fp, open(caminho_completo_anexo, 'wb') as output_fp:
            output_fp.write(attachment_fp.read())            
    
    return df_start_pipe_email, new_entry, df_trans_relation

# 1C - XXX Funcao para iniciar o pipeline atraves de outros documentos de entrada
def start_pipe_outros(df_id_relations, msg_outros, msg_outros_path, arquivos_recebidos_path):
    locale.setlocale(locale.LC_TIME, "pt_BR.utf8")
    rows_list = []
    

    for root, dirs, files in os.walk(msg_outros_path):
        doc_outros_files = [doc for doc in files]
        if doc_outros_files:
            first_doc_outros_file = doc_outros_files[0]
            file_path = os.path.join(root, first_doc_outros_file)
            msg = msg_outros
            msg_type = msg['msg_type']
            first_doc_file = first_doc_outros_file 
            new_row, new_entry, df_trans_relation = process_start_input(df_id_relations, msg, msg_type, batch_name, file_path, first_doc_file)
            rows_list.append(new_row)

    
    df_start_pipe_outros = pd.DataFrame(rows_list)
                
    return df_start_pipe_outros, new_entry, df_trans_relation

In [ ]:
# Criando registro de mensagem outras
msg_type = "mensagem_whats"

msg_outros = {
       'de': 'Daniel Nascimento',
       'dt_hora': 'qui, 10 ago 2023 23:20:56 -0300',
       'assunto': 'Teste rar',
       'email':'danielsdn0725@gmail.com',
       'msg_type': msg_type 
}
msg_outros


# 1C.1 - Iniciando o Pipeline para outras mensagens
df_start_pipe_outros, new_entry, df_trans_relation = start_pipe_outros(df_id_relations, msg_outros, msg_outros_path, arquivos_recebidos_path)
try:
    if not df_start_pipe.empty:
        df_start_pipe = df_start_pipe.append(df_start_pipe_outros, ignore_index=True)
        
        print(df_start_pipe)
except Exception as e:
        msg = (f"Erro ao buscar o df_start_pipe: {e}") 
        df_start_pipe = df_start_pipe_outros 

# Atualizando: df_id_relations
df_id_relations = df_id_relations.append(df_trans_relation, ignore_index=True)       

In [ ]:
# 1B.1 - Criando registros para documento email
df_start_pipe_email, new_entry, df_trans_relation = start_pipe_email(df_id_relations, msg_dir_path, msg_attachment_zip)
try:
    if not df_start_pipe.empty:
        df_start_pipe = df_start_pipe.append(df_start_pipe_email, ignore_index=True)
        
        print(df_start_pipe)
except Exception as e:
        msg = (f"Erro ao buscar o df_start_pipe: {e}") 
        # 1B.1 - Iniciando o Pipeline para e-mail
        df_start_pipe = df_start_pipe_email
        
# Atualizando: df_id_relations
df_id_relations = df_id_relations.append(df_trans_relation, ignore_index=True)
df_id_relations 

df_start_pipe             

In [4]:
# 2A - XXX Trata Attachments: df_attachment
def process_files(df_id_relations, i, parent_entry, batch_name, file, type, level, file_path):
    
    file_parent_name = parent_entry['File']
    file_parent_type = parent_entry['Type']
    
    parent_unique_id = get_parent_unique_id(df_id_relations, batch_name, file_parent_name, file_parent_type)
    #print(parent_unique_id)
    
    data = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)
    #df_trans_relation = df_id_relations.append(new_entry, ignore_index=True)
    
   # Busca valores processados de new_entry
    unique_id = data['Unique_ID']
    file_hash = data['Hash']
    data_process = data['Data']
    print(f'i: {i} | file: {file} | unique_id: {unique_id} ')
    document_action = "transfered_to_pipeline"
    sucess = True
    transaction_detail=" "
    src_path=""
   
    data_log = add_log_transaction_entry(unique_id, batch_name, file, document_action, src_path, file_path, transaction_detail, sucess)

    
    return {
        'index': i, 
        'Batch': batch_name,
        'Data' : data_process,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_Id': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }, data, data_log 
   
      
# 2B - Função principal para processar e-mails
def attachment_pipe(df_id_relations, parent_entry, msg_attachment_path):

    folder_file_dict = {}
    rows_list = []
    data_list = []
    log_list = []
    attachment_list = []
    arquivos_zip = []
    arquivos = []

    i = 1
    for root, dirs, files in os.walk(msg_attachment_path):
        #print(f'{root}  | {dirs} | {document} | {files}\n')
        for file in files:
            level = 2
            file_path = os.path.join(root, file)
            if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
                type = 'compressed_file_attachment'
                arquivos_zip.append(file)
            else:
                type = 'document_file_attachment'
                arquivos.append(file)
            
            
            new_row, data, data_log  = process_files(df_id_relations, i, parent_entry, batch_name, file, type, level, file_path)
            rows_list.append(new_row)
            data_list.append(data) 
            log_list .append(data_log)
            #print(f'\ni = {i}')   
        
            i += 1            

    df_attachment = pd.DataFrame(rows_list)
    df_trans_relation = pd.DataFrame(data_list)        
    df_log_tran = pd.DataFrame(log_list)

    return df_attachment, df_trans_relation, df_log_tran, attachment_list, rows_list 

In [ ]:
# 2C - Ajusto parent_entry para separar as instancias
parent_entry = new_entry

if new_entry['Type'] == "email":
    print("Estamos no caminho")
    msg_attachment_path = msg_attachment_zip
else:
    print("Outra mensagem")
    msg_attachment_path = arquivos_recebidos_path  


msg_attachment_path    

In [ ]:
# executo func attachment_pipe
df_attachment, df_trans_relation, attachment_list, rows_list = attachment_pipe(df_id_relations, parent_entry, msg_attachment_path)
df_attachment



# 2D - Atualizo: df_id_relations
df_id_relations = df_id_relations.append(df_trans_relation, ignore_index=True)
df_id_relations


# Salvando o DF para excel
df_id_relations.to_excel('df_id_relations.xlsx', index=False)
df_start_pipe.to_excel('df_start_pipe.xlsx', index=True) 

In [5]:
# 1. IMPORTANTE - Funcao de Extracao ZIP
def extract_doc_zip(trg_documento_path, documentos_extracao_path, batch_name):
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    
    zip_info = {}
    arquivos_zip = []
    i = 1
    for root, dirs, files in os.walk(trg_documento_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            level = 2
            if file.lower().endswith('.zip'):
                zip_file = file
                zip_file_path = os.path.join(root, file)
                file_type = 'compressed_file_attachment'
                print(f'\n1. Antes de buscar  | {batch_name} | file: {file} | file_type: {file_type} | level: {level}')
                l_parent_unique_id = get_document_unique_id(df_id_relations, batch_name, file, file_type, level)
                if l_parent_unique_id:
                    zip_base_name = os.path.splitext(os.path.basename(zip_file_path))[0]
                    # Cria o subdiretório com base no nome do arquivo ZIP
                    root_output_dir = os.path.join(output_dir, zip_base_name)
                    check_and_create_folder(root_output_dir)
                    # Abre o arquivo ZIP
                    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                        for member in zip_ref.namelist():
                            # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
                            parts = member.rsplit('\\', 1)
                            folder_name = parts[0] if len(parts) > 1 else ''
                            #folder_name = conv_filename(folder_temp)
                            filename = parts[-1]
                            if filename:  
                                # Cria um subdiretório se ele não existir
                                sub_dir = os.path.join(root_output_dir, folder_name)
                                sub_dir_path = os.path.normpath(sub_dir)
                                #print(f'sub_dir:  {sub_dir} | folder_name: {folder_name}')
                                check_and_create_folder(sub_dir)
                                # Salva o arquivo no subdiretório especificado
                                source = zip_ref.open(member)
                                target_path = os.path.join(sub_dir, filename)
                                # dir_path = arquivo['diretorio']
                                diretorio = os.path.basename(sub_dir_path)
                                # Verifica se a chave já existe no dicionário, senão, cria uma nova chave com uma lista como valor
                                if diretorio not in zip_info:
                                    zip_info[diretorio] = {"data": []}
                                zip_info[diretorio]["data"].append({
                                    "nome": os.path.basename(filename),
                                    "l_parent_unique_id": l_parent_unique_id,
                                    "caminho": target_path,
                                })
                                with open(target_path, "wb") as target:
                                    target.write(source.read())
                                    
    arquivos_zip.append(zip_info)
    return arquivos_zip 


# 2. IMPORTANTE, Extract RAR
def extract_doc_RAR(trg_documento_path, documentos_extracao_path, batch_name):

    output_dir = os.path.join(documentos_extracao_path, batch_name)
    # Inicialize um dicionário vazio antes de entrar no loop
    rar_info = {}
    arquivos_rar = []

    i = 1
    for root, dirs, files in os.walk(msg_attachment_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            level = 2
            if file.lower().endswith('.rar'):
                file_type = 'compressed_file_attachment'
                rar_file = file
                l_parent_unique_id = get_document_unique_id(df_id_relations, batch_name, file, file_type, level)
                if l_parent_unique_id:
                    rar_file = conv_filename_no_ext(rar_file)
                    rar_file_path = os.path.join(root, file)
                    root_output_dir = os.path.join(output_dir, rar_file)
                    check_and_create_folder(root_output_dir)
                    Archive(rar_file_path).extractall(root_output_dir)
                    # Criar um dicionário para armazenar informações sobre os arquivos extraídos
                    rar_info = {rar_file: {"data": []}}
                    # Listar todos os arquivos no diretório de saída
                    for subdir, _, files in os.walk(root_output_dir):
                        for filename in files:
                            # Obter o caminho completo do arquivo
                            full_path = os.path.join(subdir, filename)
                            # Obter o nome do diretório
                            diretorio = os.path.relpath(subdir, root_output_dir)
                            # Adicionar informações sobre o arquivo ao dicionário
                            rar_info[rar_file]["data"].append({
                                "nome": filename,
                                "caminho": full_path,
                                "l_parent_unique_id": l_parent_unique_id,
                                "diretorio": diretorio,
                            })
    arquivos_rar.append(rar_info)
    return arquivos_rar 


# 3. IMPORTANTE, Extract SEVENZ
def extract_doc_SEVENZ(trg_documento_path, documentos_extracao_path, batch_name):

    output_dir = os.path.join(documentos_extracao_path, batch_name)

    # Inicialize um dicionário vazio antes de entrar no loop
    sevenz_info = {}
    arquivos_7z = []

    i = 1
    for root, dirs, files in os.walk(msg_attachment_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            level = 2
            if file.lower().endswith('.7z'):
                file_type = 'compressed_file_attachment'
                sevenz_file = file
                l_parent_unique_id = get_document_unique_id(df_id_relations, batch_name, file, file_type, level)
                if l_parent_unique_id:
                    sevenz_file = conv_filename_no_ext(sevenz_file)
                    sevenz_file_path = os.path.join(root, file)
                    root_output_dir = os.path.join(output_dir, sevenz_file)
                    diretorio = os.path.basename(root_output_dir)
                    #check_and_create_folder(root_output_dir)
                    sevenz_info = {sevenz_file: {"data": []}}                      
                    with py7zr.SevenZipFile(sevenz_file_path, mode='r') as z:
                        z.extractall(root_output_dir)
                    #print(f'sevenz_info: {sevenz_info} | diretorio: {diretorio}\nroot_output_dir: {root_output_dir}')          
                    for subdir, _, files in os.walk(root_output_dir):
                        for filename in files:
                            print(filename)
                            # Obter o caminho completo do arquivo
                            full_path = os.path.join(subdir, filename)
                            # Obter o nome do diretório
                            diretorio = os.path.relpath(subdir, root_output_dir)
                            # Adicionar informações sobre o arquivo ao dicionário
                            sevenz_info[sevenz_file]["data"].append({
                                "nome": filename,
                                "caminho": full_path,
                                "l_parent_unique_id": l_parent_unique_id,
                                "diretorio": diretorio,
                            })
                i += 1
    arquivos_7z.append(sevenz_info)
    return arquivos_7z


# 4. IMPORTANTE, Extrac/Move PDF
def extr_move_doc_PDF(trg_documento_path, documentos_extracao_path, batch_name):

    output_dir = os.path.join(documentos_extracao_path, batch_name)
    # Inicialize um dicionário vazio antes de entrar no loop
    PDF_info = {}
    arquivos_pdf = []
    i = 1
    for root, dirs, files in os.walk(msg_attachment_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            level = 2
            if file.lower().endswith('.pdf'):
                file_type = 'document_file'
                pdf_file = file
                file_path = os.path.join(root, file)
                new_path_name = os.path.join(output_dir, file)
                diretorio = os.path.basename(file_path)
                path_relativo = file_path.replace(msg_attachment_path, "").lstrip("/")
                path_relativo_base = os.path.basename(path_relativo)
                path_relativo_dir = path_relativo.replace(path_relativo_base, "").lstrip("/")
                #check_and_create_folder(output_dir)
                try:
                    if path_relativo_dir:
                        diretorio = os.path.relpath(path_relativo_dir)
                        new_base_path = os.path.join(output_dir, diretorio)
                        new_path_name = os.path.join(new_base_path, file)
                        #print(f'\nTRUE: diretorio composto: {diretorio}\n | path_relativo_dir: {path_relativo_dir} | new_base_path_file: {new_base_path}\nnew_path_name: {new_path_name}\n')
                    else:
                        diretorio = "root"
                        #shutil.move(file_path, new_path_name)
                        #print(f'\nFALSE: diretorio: {diretorio} | file_path: {file_path}\nnew_path_name: {new_path_name}\n')
                except Exception as e:
                    print(f"Erro ao definir paths: {e}") 
                if diretorio not in PDF_info:
                    PDF_info[diretorio] = {"data": []}
                    
                PDF_info[diretorio]["data"].append({
                    "nome": pdf_file,
                    #"l_parent_unique_id": l_parent_unique_id,
                    "caminho": new_path_name,
                })
    arquivos_pdf.append(PDF_info) 
    return arquivos_pdf                                           

In [ ]:
# IMPORTANTE, Chamando a funcao para extracao de doc ZIP
extracted_docs = extract_doc_zip(msg_attachment_path, documentos_extracao_path, batch_name) 

extracted_docs = extract_doc_RAR(msg_attachment_path, documentos_extracao_path, batch_name)

extracted_docs = extract_doc_SEVENZ(msg_attachment_path, documentos_extracao_path, batch_name)

extracted_docs = extr_move_doc_PDF(msg_attachment_path, documentos_extracao_path, batch_name)


# Chama funçao para atualizar DFs
if extracted_docs:
    df_extract, df_trans_relation, df_tran_log, rows_list = extract_pipe(df_id_relations, extracted_docs)

In [ ]:

# 3A - XXX Trata documentos extraidos: df_extract (Unider Construction)
def process_extract_files(df_id_relations, i, l_parent_unique_id, batch_name, file, diretorio, type, level, file_path):
    

    #parent_unique_id = get_parent_unique_id(df_id_relations, batch_name, file_parent_name, file_parent_type)
    #print(parent_unique_id)
    parent_unique_id = l_parent_unique_id
    data = add_source_entry(batch_name, file_path, file, type, level, parent_unique_id)
    #df_trans_relation = df_id_relations.append(new_entry, ignore_index=True)
    
   # Busca valores processados de new_entry
    unique_id = data['Unique_ID']
    file_hash = data['Hash']
    data_process = data['Data']
    print(f'i: {i} | file: {file} | unique_id: {unique_id} ')
    
    document_action = "process_topipeline"
    sucess=True
    transaction_detail=" "
    src_path=" "
   
    data_log = add_log_transaction_entry(unique_id, batch_name, file, document_action, src_path, file_path, transaction_detail, sucess)
    
    
    return {
        'index': i, 
        'Batch': batch_name,
        'Data' : data_process,
        'Directory': diretorio,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_Id': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }, data, data_log
       
# 3B - Função principal para processar e-mails
def extract_pipe(df_id_relations, arquivos_zip):
    
    data_list = []
    rows_list = []
    log_list = []
    i = 1
    for zip_info in arquivos_zip:
        for diretorio, dados in zip_info.items():
            print(f'Diretório (ZIP): {diretorio}\n')
            for arquivo in dados['data']:
                # print(f'    Arquivo: {arquivo["nome"]}')
                # print(f'    {arquivo["caminho"]}\n\n')
                file_path = arquivo["caminho"]
                file = arquivo["nome"]
                l_parent_unique_id = arquivo["l_parent_unique_id"]
                
                level = 3
                type = 'document_file'
                new_row, data, data_log = process_extract_files(df_id_relations, i, l_parent_unique_id, batch_name, file, diretorio, type, level, file_path)
                rows_list.append(new_row)
                data_list.append(data)
                log_list.append(data_log) 
                #print(f'\ni = {i}')   
        
                i += 1            
    df_tran_log = pd.DataFrame(log_list)
    df_extract = pd.DataFrame(rows_list)
    df_trans_relation = pd.DataFrame(data_list)        


    return df_extract, df_trans_relation, df_tran_log, rows_list


# Teremos que olhar isto para o 3
    file = os.path.basename(file_path)

    if (document_type == "compressed_file_attachment") and document_level == 2:
        print(f'E documento e um attachment ou ZIP, RAR ou 7Z: {document_file}')
    elif (document_type == "document_file_attachment") and document_level == 2:
        if document_file.lower().endswith('.pdf'):
            get_father(batch, file_path)
            print(f'Documento é um attachment de: , embora seja um PDF: {document_file}')

In [ ]:
# IMPPORTANTE _ OFICIAL df_log_transaction Tratamento dos DFss de log 
df_log_transaction = pd.DataFrame()
df_log_tran = pd.DataFrame()
df_row = pd.DataFrame()

df_log_transaction = df_log_transaction.append(df_row, ignore_index=True)

In [ ]:
# Importante para consultas
src_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos/fwdnotasemitidasem31072023.zip"
file = os.path.basename(file_path)
document_unique_id = get_document_id_by_file(batch_name, file)

In [ ]:
# Importante para geraçao de novo MOVE
src_path = "pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos/fwdnotasemitidasem31072023.zip"
tgt_path = "pipeline_extracao_documentos/5_documentos_processados"


df_row, sucesso = move_doc_processed_file(batch_name, src_path,  tgt_path)

## 2. POC Pre-process

### 2.1 - Modulos, configs, dicts e funcoes desta parte

In [6]:
import modules.extrai_pdf_pesquisavel as Extc

In [7]:
# CONFIGS DESTA PARTE
# 1. Configuraçoes de pastas
documentos_scan_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"

### Template de modelos

In [8]:
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"


#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']


In [ ]:
frames_nf_v4_df

<mark> <b>1.</b> Consistencia de Modelos Prefeitura x CNPJ Prestador </mark>

In [9]:
def pesquisa_dict_prefeitura_modelo(texto):

    # 7. ZZZ Dicionário para mapear Prefeitura e/ou CNPJ para um template específico
    templates = {
        ("PREFEITURA DA CIDADE MAGE", "30.693.231/0001-99"): "MAGE_MAICON",
        ("PREFEITURA DA CIDADE MAGE", "23.317.112/0001-76"): "MAGE_MFF",
        ("PREFEITURA DA CIDADE MAGE", None): "MAGE",
        ("PREFEITURA MUNICIPAL DE MAGE", None): "MAGE",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "47.945.459/0001-21"): "SAO_PEDRO_GOAT",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "68.687.722/0001-08"): "SAO_PEDRO_GM",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", "34.230.979/0038-06"): "SAO_PEDRO_SUPERMIX",
        ("PREFEITURA MUNICIPAL DE SAO PEDRO DA ALDEIA", None): "SAO_PEDRO",
        ("Pague agora com o seu Pix", None): "NAO_PROCESSAR",
        # ... adicione outras combinações aqui
    }




    # 8. ZZZ Consistencia para buscar modelo
    prefeitura_encontrada = None
    cnpj_encontrado = None

    # Verifique cada linha do texto
    for linha in texto:
        for pref, cnpj in templates.keys():
            if pref in linha:
                #print(linha)
                prefeitura_encontrada = pref
            if cnpj and cnpj in linha:
                cnpj_encontrado = cnpj

    # Saímos do loop, agora vamos verificar qual template usar
    if prefeitura_encontrada:
        template_usar = templates.get((prefeitura_encontrada, cnpj_encontrado))
        if not template_usar:
            template_usar = templates.get((prefeitura_encontrada, None), "NAO_PROCESSAR")
    else:
        template_usar = "NAO_PROCESSAR"

    return template_usar, prefeitura_encontrada, cnpj_encontrado

<h3><mark> <b>1.</b> Funcoes importantes </mark></h3>

### FUNCOES DE TRATAMENTO QUALIFICADAS

In [10]:
# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 


# XXX Funcao ajustada para convertere e resize
def convertResize(doc2convert, document_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(doc2convert)
    
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
        image_2work = resized_pages[0]
        
    return image_2work, image_resized_name

### Funcoes de OCR

In [11]:
import os
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path

import pytesseract

poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

In [ ]:

# Trata Ocr
def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame_2(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores


# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

 
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text 


def extract_fb_outras_inf(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == model)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        
        string_pesquisa = row_field['reference']
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        label = row_field['label']
        #print(f'extracted_text_box {extracted_text_box}, label {label}')
        text = extracted_text_box.replace('\n', '')
        if text.startswith(string_pesquisa):
            #print("aqui:", text)
            text = text[len(label):].strip()
            data_box_valores[label] = text
    
    return   data_box_valores  


### Funcoes REGEX e ORganizacao TEXTO

In [12]:
# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None
    
    
# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)



#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  


    
def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):

    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz           
    #print(resultado_extraido)
    return resultado_extraido



def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf   





def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj 



def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao 
            

#4. Extrai prefeitura
def extract_prefeitura(model, father, values):
    
    tipo = "sframe_field"
    data_extrated_prefeitura = {}
    #print(tipo)

    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

    for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
        
        label_value = row_sframe['label']
        
        #print("label_value", label_value)
        
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura


def processa_cnae_outros(text):
    nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
    if nf_data_CNAE_match:
        try:
            # Remove a primeira ocorrência de "CNAE:"
            nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
            # Remover quebras de linha
            nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
            return nf_data_CNAE_str 
        except Exception as e:
            print(f"Erro busca cnae: {e}") 
        
    return None   





### Funcoes de Tratamento do Model

In [ ]:
# 1. XXX Avaliar Funçao basica de modelo 
def pesquisa_model_frame_coordenadas(model_src, tipo, tipo_processo_pdf):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_prefeitura = row_frame['prefeitura']
        frame_label = row_frame['label']
        frame_section_json = row_frame['section_json']
        frame_cnpj = row_frame['cnpj']
        
        if frame_model:
            modelo = True
        
        if tipo_processo_pdf == 'pdf_pesquisavel':
            try:
                # data for PDF Pesquisavel
                x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)   
        
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                

        elif tipo_processo_pdf == 'raster_pdf':
            try:
                # data for Raster_PDF
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                
                data_dados_model = {
                    "prefeitura": frame_prefeitura,
                    "model": frame_model,
                    "label": frame_label,
                    "section_json": frame_section_json,
                    "cnpj": frame_cnpj,
                    "x0": x0,
                    "y0": y0,
                    "x1": x1,
                    "y1": y1,
                }
                dados_modelo.append(data_dados_model)
            except Exception as e:
                print(f"Erro ao carregar coordenadas do frame: {e}")
                x0, y0, x1, y1 = 0, 0, 0, 0 
                  
  
    return dados_modelo, modelo


# 1. funçao basica de modelo 
def pesquisa_model_frame(model_src, tipo):

    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        # frame_prefeitura = row_frame['prefeitura']
        # frame_label = row_frame['label']
        # frame_section_json = row_frame['section_json']
        # frame_cnpj = row_frame['cnpj']
        
        # if frame_model:
        #     modelo_frame = True
        # else:    
        #     modelo_frame = False

                  
  
    return frame_model

### <mark> Funçoes de Extracao Qualificadas </mark>

In [ ]:

 



def remove_images(imagens_list):
    for image in imagens_list:
        try:
            os.remove(image)
        except Exception as e:
            print(f"Nao congui deletar: {e}")

In [ ]:
# 1. XXX funçao basica de modelo 
def pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label):

    tipo = tipo
    data_dados_model = {}
    dados_modelo = []
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model_src) & (frames_nf_v4_df['type'] == tipo) & (frames_nf_v4_df['section_json'] == section) & (frames_nf_v4_df['label'] == label)]
    
    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        frame_model = row_frame['model']
        frame_label = row_frame['label']
 
        # data for PDF Pesquisavel
        x0, y0, x1, y1 = row_frame['x0_p'], row_frame['y0_p'], row_frame['x1_p'], row_frame['y1_p']
 
    return x0, y0, x1, y1

### <mark> <b>2.</b> Funcoes de tratamento de texto para cabecalho </mark>

In [14]:
def extract_text_PIL(image, coordinates):
    x0, y0, x1, y1 = coordinates
    image_croped = image.crop((x0, y0, x1, y1))
    texto_extraido = pytesseract.image_to_string(image_croped, lang='por')
    return texto_extraido 




# XXX Funcao de extracao
def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    print(x0, y0, x1, y1)
    frame_image
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def extract_text_from_coordinates_2(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    print(x0, y0, x1, y1)
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config_1).strip()
    return extracted_text

In [ ]:
# 3B. XXX Efetuo a extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'extracao_prestador' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'OK'

#raw_texto = extracao_pipeline(df_analise_pipe, fase, atividade, status)
raw_texto = extracao_pipeline(subset_df_analise_pipe, fase, atividade, status)

In [ ]:
# 1. XXX funçao IMPORTANTE basica de modelo 
def executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work):

    data_dados_frame = {}
    
    print(f'3 em executa model_frame - antes do for {model} se: {secao} |f_frame_name: {f_frame_name} | f_tipo: {f_tipo}')
    
    
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['label'] == f_frame_name) & (frames_nf_v4_df['type'] == f_tipo)]

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        texto_extraido_pil = extract_text_PIL(image_2work, (x0, y0, x1, y1)) # PONTO IMPORTANTE
        print(f'3 - executa: {model} | {secao} | {f_frame_name}  x0: {x0} y0: {y0} x1: {x1} y1: {y1}')
        
        frame_seq = row_frame['seq']
        frame_model = row_frame['model']
        frame_label = row_frame['label']
        frame_type = row_frame['type']
        frame_section = row_frame['section_json']
        frame_reference = row_frame['reference']
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        #print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
        
    return texto_extraido_pil
  
  
  
def extract_fields_cabecalho_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    
    model = row['modelo']   
    secao = "1 - CABECALHO"
    section = "1 - CABECALHO"
    f_frame_name = "1_frame_dados_nf"
    f_tipo = 'frame'
        
    raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) # executa_model_frame
    
    
    try:
        text_splited = texto_extraido_cabecalho(raw_texto_pil)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
        
    except Exception as e:
        # erros_cabecalho = {}
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
        
        
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section, raw_texto_pil 

def extract_fields_cabecalho_pdf_pesquisavel(row, file_path):
    
    message_erro = [] 
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    
    
    model_src = row['modelo']
    original_file_name = row['original_file_name']
    tipo = "frame"
    section = '1 - CABECALHO'
    label = '1_frame_prefeitura_nf'

    x0, y0, x1, y1 = pesquisa_frame_coordenadas_pdf_pesquisa(model_src, tipo, section, label)

    try:
        texto = page.get_text("text", clip=(x0, y0, x1, y1))
        
        text_splited = texto_extraido_nf(texto)
        keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

        string_pesquisa = "Número da Nota:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
        nro_nota_frame = texto
        
        string_pesquisa = "Competência:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        competencia_frame = texto
        
        string_pesquisa = "ata e Hora da Emissão:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        dt_hr_emissao_frame = texto
        
        string_pesquisa = "Código Verificação:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        codigo_verificacao_frame = texto
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        nro_nota_frame = " "
        competencia_frame = " "
        dt_hr_emissao_frame = " "
        codigo_verificacao_frame = " "
        message_erro.append(msg)
            
    pdf_document.close()
    #print(row)
    return nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section


# 5. XXX Ajusta texto
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

# 6. XXX Ajusta texto para PDF_Pesquisavel-  NO CABECALHO
def texto_extraido_nf(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


# 6. XXX Ajusta texto para PDF RASTER NO CABECALHO
def texto_extraido_cabecalho(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    text_splited = [s.replace(")", "").strip() for s in text_splited] #depende da situaçao
    return text_splited



### <mark> <b>2.</b> Teste de valores de Texto para Cabecalho </mark>

In [ ]:
import re

def corrigir_erro_ocr(texto):
    
    # Mapeamento de correções comuns
    correcoes = {
        "B": "8",
        "I": "1",
        "l": "1", # l minúsculo pode ser confundido com 1
        "S": "5",
        # Adicione mais correções conforme necessário
    }

    # Aplicar as correções
    for errado, correto in correcoes.items():
        texto = texto.replace(errado, correto)
    
    # Correções específicas para padrões identificados
    texto = re.sub(r'0O0', '000', texto) # Substitui '0O0' por '000'
    
    # Se o texto tem 10 caracteres, tentamos remover cada caracter para chegar a uma string de 9 caracteres
    if len(texto) == 10:
        for i in range(10):
            texto_temp = texto[:i] + texto[i+1:]
            if len(texto_temp) == 9:
                texto = texto_temp
                break
    
    # Certificando que o texto tem o comprimento correto
    if len(texto) != 9:
        return "Erro: comprimento incorreto"
    
    return texto


In [ ]:
# Testando a função
print(corrigir_erro_ocr("7C0O0AE077")) # Deveria corrigir para "7C000AE07"

In [ ]:
string = "7C0O0AE07"

In [ ]:
len(string)

In [ ]:
corrigir_erro_ocr(string)

In [ ]:
def corrigir_erro_ocr(texto):
   
    if len(texto) != 9:
        msg = ("Erro: comprimento incorreto")

    # Mapeamento de correções comuns
    correcoes = {
        "B": "8",
        "O": "0",
        "I": "1",
        "l": "1", # l minúsculo pode ser confundido com 1
        "S": "5",
        
    }

    # Aplicar as correções
    for errado, correto in correcoes.items():
        texto = texto.replace(errado, correto)
    
    return texto

In [ ]:
codigo_verificacao_frame

In [ ]:
texto = corrigir_erro_ocr(codigo_verificacao_frame)

In [ ]:
codigo_verificacao_frame

In [ ]:
len(codigo_verificacao_frame)

In [ ]:
len(texto)

In [ ]:
print(corrigir_erro_ocr("7CO00AE077")) # Deveria corrigir para "7C000AE077"

In [ ]:
text_splited = texto_extraido_cabecalho(raw_texto)
text_splited

In [ ]:
keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

string_pesquisa = "Número da Nota:"
texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
nro_nota_frame = texto

nro_nota_frame

In [ ]:
string_pesquisa = "Competência:"
texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
competencia_frame = texto
competencia_frame

In [ ]:
string_pesquisa = "ata e Hora da Emissão:"
texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
dt_hr_emissao_frame = texto
dt_hr_emissao_frame

In [ ]:
string_pesquisa = "Código Verificação:"
texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
codigo_verificacao_frame = texto
codigo_verificacao_frame

## <mark> Analise de Extracao de dados no Pipeline </mark>

In [ ]:
# 2. XXX Analisa nro de paginas
def analisa_nro_pages(file_path):
    
    pdf_document = fitz.open(file_path)
    pages = pdf_document.pages() # generator object

    page_nro = []
    for page in pages:
        page_nro.append(page)
        
    nro_paginas = len(page_nro)    
    if nro_paginas > 1:
        doc_1_page = False
        return doc_1_page, nro_paginas    
    else:
        doc_1_page = True
        return doc_1_page, nro_paginas  
    pdf_document.close()
    
 

# XXX FUNCAO DE SPLIT
def split_documentos(qualquer_df, outro_tgt_df, num_linhas_df, fase, atividade, status):
    
    documentos_splitados = []
    doc_info = {}
    rows_list = []
    documentos = []
    #output_dir = os.path.join(documentos_scan_path, batch_name)

    fase_processo = fase
    atividade_processo = atividade
    status_documento = status
    
    time_now = cron.timenow_pt_BR()
    
    i = num_linhas_df + 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        batch_name = row['batch']
        original_file_name = row['original_file_name']
        folder_name = row['directory']
        file_path = row['file_path']
        level = row['level']
        d_type = row['type']
        document_type = row['document_type']
        # document_unique_id = row['document_unique_id']
        new_level = level + 1
        try:
            pdf = fitz.open(file_path)
            # Número total de páginas no PDF
            total_pages = len(pdf)
        except Exception as e:
            print(f"Nao congui abrir o PDF: {e}")    

        # Nome base para os arquivos de saída
        base_name = file_path.split('.')[0]  # Remove a extensão do arquivo
        
        file_to_delete = file_path

        # Loop para criar um novo PDF para cada página
        for page_num in range(total_pages):
            # Cria um novo objeto PDF
            new_pdf = fitz.open()
            # Adiciona a página atual ao novo PDF
            new_pdf.insert_pdf(pdf, from_page=page_num, to_page=page_num)
            # Nome do novo arquivo PDF
            new_pdf_name = f"{base_name}_page_{page_num + 1}.pdf"
            # Salva o novo PDF
            new_pdf.save(new_pdf_name)
            # Fecha o novo PDF
            new_pdf.close()
            status_documento = 'splitado'
            name_pdf_splited = os.path.basename(new_pdf_name)
            
            nova_linha = {
                'seq': i,
                'date_time': time_now,
                'batch': batch_name,
                'fase_processo': fase_processo,
                'nome_atividade': atividade_processo,
                'status_documento': status_documento,
                'document_unique_id': generate_unique_id(),
                'original_file_name': name_pdf_splited,
                'directory': folder_name,
                'one_page': True,
                'pages': 1,                    
                'level': new_level,
                'type': d_type,
                'document_type': document_type,
                'parent_document_unique_id': idx,
                'file_path': new_pdf_name,
                'file_hash': generate_file_hash(new_pdf_name),
            }
            rows_list.append(nova_linha)
            i += 1
        outro_tgt_df.loc[idx, 'status_documento'] = "NAO_PROCESSAR"    
            

    total_split = i - 1
                      
      
    df_split = pd.DataFrame(rows_list)
    
    return df_split


# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos
def scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status):

    doc_info = {}
    resumo = {}
    documentos = []
    output_dir = os.path.join(documentos_scan_path, batch_name)

    fase_processo = fase
    atividade_processo = atividade
    status_documento = status
    
    time_now = cron.timenow_pt_BR()
    
    #print(f'1. Fase atual: {fase_atual} | Nome atividade: {nome_atividade} | Status documento: {status_documento}')
    
    # Just for DEV
    parent_document_unique_id = generated_parent_document_unique_id

    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            level = 3
            file_path = os.path.join(root, file)
            new_path_name = os.path.join(output_dir, file)
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
            doc_one_page, nro_pgs = analisa_nro_pages(file_path)
            #print(f'2. doc_one_page: {doc_one_page} | nro_pgs: {nro_pgs}') 
            type = 'document_file'
            document_type = "provavel_NFSe"
            #print(f'\nfile: {file} | diretorio: {folder_name}\n{file_path} ')
            if folder_name not in doc_info:
                doc_info[folder_name] = {"data": []}
            doc_info[folder_name]["data"].append({
                "seq": i,
                "date_time": time_now,
                "batch": batch_name,
                "fase_processo": fase_processo,
                "nome_atividade": atividade_processo,
                "original_file_name": file,
                "status_documento": status_documento,
                "one_page": doc_one_page,
                "pages": nro_pgs,
                "directory": folder_name,
                "file_path": file_path,
                "level": level,
                "type": type,
                "document_type": document_type,
                "document_unique_id": generate_unique_id(),
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": generate_file_hash(file_path)
            })

            i += 1

    
    documentos.append(doc_info)
    total_scan = i - 1
    #print("Documentos scaneados: ", total_scan)
    
    # Criaçao do df_scan_pipe
    document_row = {}
    rows_list = []
    for documento in documentos:
        #print(documento)
        for diretorio, dados in documento.items():
            #print(f'Diretório: {diretorio}\n')
            for arquivo in dados['data']:
                nova_linha = {
                    'seq': arquivo['seq'],
                    'date_time': arquivo['date_time'],
                    'batch': arquivo['batch'],
                    'fase_processo': arquivo['fase_processo'],
                    'nome_atividade': arquivo['nome_atividade'],
                    'status_documento': arquivo['status_documento'],
                    'document_unique_id': arquivo["document_unique_id"],
                    'original_file_name': arquivo['original_file_name'],
                    'directory': arquivo['directory'],
                    'one_page': arquivo['one_page'],
                    'pages': arquivo['pages'],
                    'pdf_pesquisavel': None,
                    'prefeitura': None,
                    'cnpj': None,
                    'modelo': None,
                    'level': arquivo['level'],
                    'type': arquivo['type'],
                    'document_type': arquivo['document_type'],
                    'parent_document_unique_id': arquivo['parent_document_unique_id'],
                    'file_path': arquivo["file_path"],
                    'file_hash': arquivo["file_hash"],
                }
                rows_list.append(nova_linha)
    
    df_trans_pipe = pd.DataFrame(rows_list)            
    
    return df_trans_pipe, documentos



    
    
#generated_parent_document_unique_id = generate_unique_id()  

# Processo de deleçao e atualizacao de documentos
#e_deleta_peloamor(df_docs_splitados)

#me_atualiza_logo_vai_2(novo_df)

In [ ]:
# 0. XXX SE deseja importar o DF df_analise_pipe

df_analise_pipe_path = "df_mapeamento_e_analise0.xlsx"


#Le a planilha e cria df_documento_recebido
df_analise_pipe = pd.read_excel(df_analise_pipe_path)

# Ajusta o indice
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
df_analise_pipe

<mark> <b>2.x</b> Processo de mapeamento e criacao do  df_analise_pipe </mark>

In [ ]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'mapear'

df_analise_pipe, documentos = scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status)


# 2. XXX df_analise_pipe e o df oficial de analise do pipeline
df_analise_pipe

# 3. Numero de linhas do DF
num_linhas_df = df_analise_pipe.shape[0]

num_linhas_df # variavel para o numero de linhas do DF


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_analise_pipe[df_analise_pipe['one_page'] == False]

num_docs_2_split = df_pages_2_split.shape[0]


print(f'nro documentos em df_analise_pipe: {num_linhas_df} | nro documentos + 1 pagina: {df_pages_2_split.shape[0]}')
    
# 5. XXX Definimos o Index dos DFs
df_analise_pipe.set_index('document_unique_id', inplace=True)

df_pages_2_split.set_index('document_unique_id', inplace=True)



# 6. XXX Executo a criacao dos documentos split 
fase = 'pre-processamento'
atividade = 'scan_analise'
status = 'processar'

# df_docs_splitados recebera o DF com os documentos splitados
df_docs_splitados = split_documentos(df_pages_2_split, df_analise_pipe, num_linhas_df, fase, atividade, status)



# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)

# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)

# 10. XXX Salvo em Excel (pode ser feito durante fases)
df_analise_pipe.to_excel("df_mapeamento_e_analise.xlsx", index=True)

<h3><mark> <b>2.</b> Analise e extracao de dados no pipeline </mark></h3>

In [ ]:
# Esta e o principio da melhor funcao do mundo
def analise_extracao_pipeline(qualquer_df, outro_tgt_df, fase, atividade, status):
    
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []
    imagens_list = []   
    
    pre_processo = ['ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    time_now = cron.timenow_pt_BR()
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        seq_df = row['seq']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        pdf_pesquisavel = row['pdf_pesquisavel']
        one_page_doc = row['one_page']
        
        prefeitura = row['prefeitura']
        
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # 2. Busca modelo
        if (not status_documento == 'NAO_PROCESSAR') or (not document_type == 'outros') or (one_page_doc == True):
            
            # 1. Analisa se PDF e pesquisavel
            if atividade_processo_atual == 'analisar_pdf_pesquisavel':
                
                pdf_realmente_pequisavel, page_number = confirma_pdf_pequisavel3(file_path)
                #if pdf_realmente_pequisavel:
                    
                qualquer_df.loc[idx, 'date_time'] = time_now
                outro_tgt_df.loc[idx, 'date_time'] = time_now 
                
                #print(f'seq: {seq_df} | pdf_pesquisavel: {pdf_realmente_pequisavel} | file: {original_file_name}')
                qualquer_df.loc[idx, 'pdf_pesquisavel'] = pdf_realmente_pequisavel
                outro_tgt_df.loc[idx, 'pdf_pesquisavel'] = pdf_realmente_pequisavel
                
                    
            # 2. Busca prefeitura
            elif atividade_processo_atual == 'pesquisar_prefeitura':
                
                if pdf_pesquisavel == True:
                    text = pesquisa_prefeitura_pdf_pesquisavel(file_path)
                    texto = texto_extraido_nf(text)
                    
                    template_usar, prefeitura_encontrada, cnpj_encontrado = pesquisa_dict_prefeitura_modelo(texto)
                    
                    if template_usar != 'NAO_PROCESSAR':
                        
                        model_src = template_usar
                        tipo = "document"
                        frame_model_pesquisado = pesquisa_model_frame(model_src, tipo)
                        model_frame = True
                        
                        outro_tgt_df.loc[idx, 'date_time'] = time_now
                        qualquer_df.loc[idx, 'date_time'] = time_now 
                        
                        qualquer_df.loc[idx, 'prefeitura'] = prefeitura_encontrada
                        outro_tgt_df.loc[idx, 'prefeitura'] = prefeitura_encontrada
                        
                        qualquer_df.loc[idx, 'modelo'] = frame_model_pesquisado
                        outro_tgt_df.loc[idx, 'modelo'] = frame_model_pesquisado
                        
                        qualquer_df.loc[idx, 'model_frame'] = model_frame
                        outro_tgt_df.loc[idx, 'model_frame'] = model_frame
                        
                        qualquer_df.loc[idx, 'cnpj'] = cnpj_encontrado
                        outro_tgt_df.loc[idx, 'cnpj'] = cnpj_encontrado
                        
                        #print(f'template_usar: {template_usar} | frame_model_pesquisado{frame_model_pesquisado} | file_name: {original_file_name} ')
                        
                    else:
                        #print(f'\ntemplate_usar: {template_usar}  | file_name: {original_file_name}\n') 
                        model_frame = False
                        mensagem_status = "Reavaliar_PDF_Pesquisavel"
                        
                        qualquer_df.loc[idx, 'modelo'] = "Nao Econtrato"
                        outro_tgt_df.loc[idx, 'modelo'] = "Nao Econtrato"
                        
                        qualquer_df.loc[idx, 'model_frame'] = model_frame
                        outro_tgt_df.loc[idx, 'model_frame'] = model_frame
                        
                        
                        qualquer_df.loc[idx, 'status_documento'] = mensagem_status
                        outro_tgt_df.loc[idx, 'status_documento'] = mensagem_status
                        
                        #print(f'Nao ha template para este documento: {template_usar} model_frame: {model_frame} ')   

                        
                    # if model_frame:
                        
            elif atividade_processo_atual == 'Reavaliar_PDF_Pesquisavel':
                
                if status_documento == 'Reavaliar_PDF_Pesquisavel':
                
                    image_on_the_fly, image_resized_name = convertResizeAnalise_1page(original_file_name, file_path, image_resized_path)
                    extracted_text_frame = pequisaTextoDoc(image_on_the_fly)
                    texto = texto_extraido_nf(extracted_text_frame)
                    
                    template_usar, prefeitura_encontrada, cnpj_encontrado = pesquisa_dict_prefeitura_modelo(texto)
                    imagens_list.append(image_resized_name)
                    
                    if template_usar != 'NAO_PROCESSAR':
                        
                        model_src = template_usar
                        tipo = "document"
                        frame_model_pesquisado = pesquisa_model_frame(model_src, tipo)
                        model_frame = True
                        mensagem_status = "Template_encontrado"
                        
                        outro_tgt_df.loc[idx, 'date_time'] = time_now
                        qualquer_df.loc[idx, 'date_time'] = time_now 
                        
                        qualquer_df.loc[idx, 'prefeitura'] = prefeitura_encontrada
                        outro_tgt_df.loc[idx, 'prefeitura'] = prefeitura_encontrada
                        
                        qualquer_df.loc[idx, 'modelo'] = frame_model_pesquisado
                        outro_tgt_df.loc[idx, 'modelo'] = frame_model_pesquisado
                        
                        qualquer_df.loc[idx, 'model_frame'] = model_frame
                        outro_tgt_df.loc[idx, 'model_frame'] = model_frame
                        
                        qualquer_df.loc[idx, 'cnpj'] = cnpj_encontrado
                        outro_tgt_df.loc[idx, 'cnpj'] = cnpj_encontrado
                        
                        qualquer_df.loc[idx, 'status_documento'] = mensagem_status
                        outro_tgt_df.loc[idx, 'status_documento'] = mensagem_status
                        
                        qualquer_df.loc[idx, 'pdf_pesquisavel'] = False
                        outro_tgt_df.loc[idx, 'pdf_pesquisavel'] = False
                        
                        #print(f'template_usar: {template_usar} | frame_model_pesquisado{frame_model_pesquisado} | file_name: {original_file_name} ')
                        
                    else:
                        #print(f'\ntemplate_usar: {template_usar}  | file_name: {original_file_name}\n') 
                        model_frame = False
                        mensagem_status = "Reavaliar_Tipo_Documento"
                        
                        qualquer_df.loc[idx, 'modelo'] = "Nao Econtrato"
                        outro_tgt_df.loc[idx, 'modelo'] = "Nao Econtrato"
                        
                        qualquer_df.loc[idx, 'model_frame'] = model_frame
                        outro_tgt_df.loc[idx, 'model_frame'] = model_frame
                        
                        
                        qualquer_df.loc[idx, 'status_documento'] = mensagem_status
                        outro_tgt_df.loc[idx, 'status_documento'] = mensagem_status
                    
                    #print(f'seq: {seq_df} | file: {original_file_name} | atividade: {atividade_processo_atual}\n{template_usar} | {prefeitura_encontrada}\n\n')
                    
                # elif pdf_pesquisavel == False:    
                #     image_on_the_fly, image_resized_name = convertResizeAnalise_1page(original_file_name, file_path, image_resized_path)
                #     imagens_list.append(image_resized_name)
                #     nome_prefeitura = pequisaModel(image_on_the_fly)
                # else:
                #     print(f' valor pdf_pesquisavel: {pdf_pesquisavel} | file: {original_file_name} - sugiro executar a atividade: analisar_pdf_pesquisavel')    
                    
            
                # if nome_prefeitura:
                #     print(f'\n\nSUCESSO: seq: {seq_df} | file: {original_file_name}   | nome_prefeitura: {nome_prefeitura} | status_documento: {status_documento}\n\n')
                #     new_document_type = "provavel_NFSe"
                #     new_status_documento = "PROCESSAR"
                    
                    # outro_tgt_df.loc[idx, 'date_time'] = time_now
                    # qualquer_df.loc[idx, 'date_time'] = time_now  
                    
                    # qualquer_df.loc[idx, 'prefeitura'] = nome_prefeitura
                    # outro_tgt_df.loc[idx, 'prefeitura'] = nome_prefeitura
                
                
                
                # outro_tgt_df.loc[idx, 'document_type'] = new_document_type
                # outro_tgt_df.loc[idx, 'status_documento'] = new_status_documento
                
                # else:
                #     print(f'\nNAO ACHEI: seq: {seq_df} | file: {original_file_name}   | nome_prefeitura: {nome_prefeitura} | status_documento: {status_documento}')
                #     new_document_type = "outros"
                #     new_status_documento = "NAO_PROCESSAR"
                
                # outro_tgt_df.loc[idx, 'date_time'] = time_now    
                # outro_tgt_df.loc[idx, 'document_type'] = new_document_type
                # outro_tgt_df.loc[idx, 'status_documento'] = new_status_documento  
                
                # qualquer_df.loc[idx, 'date_time'] = time_now 
                # qualquer_df.loc[idx, 'document_type'] = new_document_type
                # qualquer_df.loc[idx, 'status_documento'] = new_status_documento
            
            
            
            
            

        

     
        i += 1
    total_map = i - 1
    print()
    print("Documentos mapeados: ", total_map)
    
    return imagens_list

In [ ]:
# 2B. XXX Efetuo a analise do pipeline de documentos e inicio da extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'Reavaliar_PDF_Pesquisavel' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'mapear'

imagens_list = analise_extracao_pipeline(subset_df_analise_pipe, df_analise_pipe, fase, atividade, status)


if imagens_list:
    remove_images(imagens_list)

In [ ]:
def extract_fields_inscricao_prestador_raster_pdf(row, file_path, image_2work): 
    
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    model = row['modelo']   
   
    secao = "2. PRESTADOR DE SERVIÇO"
    section = secao
     
    try:
        f_frame_name = "2_frame_inscricao_prestador" 
        print(f'1 no try da insciao :  {model} se: {secao} |f_frame_name: {f_frame_name} | f_tipo: {f_tipo}')
        raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) 
        text_splited = texto_extraido(raw_texto_pil)
        keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
        string_pesquisa = "Inscrição Municipal:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        inscricao_municipal_prestador_frame = texto
        
        string_pesquisa = "Inscrição Estadual:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        inscricao_estadual_prestador_frame = texto
      
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        inscricao_municipal_prestador_frame = " "
        inscricao_estadual_prestador_frame = " "
        raw_texto_pil = ' '
        
    return inscricao_municipal_prestador_frame, inscricao_estadual_prestador_frame, msg, raw_texto_pil   

In [ ]:
def extract_fields_prestador_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    model = row['modelo']   
   
    secao = "2. PRESTADOR DE SERVIÇO"
    section = secao
    try:
        f_frame_name = "2_frame_cnpj_prestador"
        
        raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) 
        
        # Extrair CPF/CNPJ com máscara 1
        if "CPF/CNPJ:" in raw_texto_pil:
            cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', raw_texto_pil)
            if cpf_cnpj_formatado_match:
                cpf_cnpj_com_mascara_prestador = cpf_cnpj_formatado_match.group(1)
                cpf_cnpj_sem_mascara_prestador = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
        else:
            cpf_cnpj_com_mascara_prestador = None
            cpf_cnpj_sem_mascara_prestador = None            
            
            
        telefone_str = None
    
        #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
        telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', raw_texto_pil)
        if telefone_match: 
            telefone_str = telefone_match.group(1)
            # Remover quebras de linha
            telefone_str = telefone_str.replace('.', '')
            telefone_str = telefone_str.replace('\n', '')
                    
            telefone_prestatador_frame = telefone_str
        else:
            telefone_prestatador_frame = None 
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        cpf_cnpj_com_mascara_prestador = None
        cpf_cnpj_sem_mascara_prestador = None
        telefone_prestatador_frame = None  
             
        
    try:
        f_frame_name = "2_frame_inscricao_prestador" 
        
        raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) 
        text_splited = texto_extraido(raw_texto_pil)
        keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
        string_pesquisa = "Inscrição Municipal:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        inscricao_municipal_prestador_frame = texto
        
        string_pesquisa = "Inscrição Estadual:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        inscricao_estadual_prestador_frame = texto
      
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        inscricao_municipal_prestador_frame = " "
        inscricao_estadual_prestador_frame = " "

  
    try:
        f_frame_name = "2_frame_dados_prestador"
        f_tipo = 'frame'
        
        
        raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work)
        
        keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
        
        string_pesquisa = "Nome/Razão Social:"
        text_splited = texto_extraido(raw_texto_pil)
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        razao_social_prestador_frame = texto

        string_pesquisa = "Nome de Fantasia:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        nome_fantasia_prestador_frame = texto
        
        string_pesquisa = "Endereço:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        endereco_prestador_frame = texto
        
        string_pesquisa = "E-mail:"
        texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
        email_prestador_frame = texto
        
    except Exception as e:
        msg = (f"doc: {original_file_name} | {e}")
        razao_social_prestador_frame = " "
        nome_fantasia_prestador_frame = " "
        endereco_prestador_frame = " "
        email_prestador_frame = " "
        
    return cpf_cnpj_com_mascara_prestador, cpf_cnpj_sem_mascara_prestador, telefone_prestatador_frame, inscricao_municipal_prestador_frame, inscricao_estadual_prestador_frame, razao_social_prestador_frame, nome_fantasia_prestador_frame, endereco_prestador_frame, email_prestador_frame



In [ ]:
def extract_fields_cnpj_prestador_raster_pdf(row, file_path, image_2work): 
  
    message_erro = []
  
  
    model_src = row['modelo']
    original_file_name = row['original_file_name'] 
    model = row['modelo']  
    
    f_tipo = 'frame' 
   
    secao = "2. PRESTADOR DE SERVIÇO"
    section = secao
    print(f'1- antes do try {model} se: {secao} | f_tipo: {f_tipo}')
    
    f_frame_name = "2_frame_cnpj_prestador"
    
    
    
    raw_texto_pil = executa_model_frame(model, secao, f_frame_name, f_tipo, image_2work) 
    
    print(f'2 dentro do try {model} se: {secao} |f_frame_name: {f_frame_name} | f_tipo: {f_tipo} | raw_texto_pil: {raw_texto_pil}')
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in raw_texto_pil:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', raw_texto_pil)
        if cpf_cnpj_formatado_match:
            cpf_cnpj_com_mascara_prestador = cpf_cnpj_formatado_match.group(1)
            cpf_cnpj_sem_mascara_prestador = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))
    else:
        cpf_cnpj_com_mascara_prestador = None
        cpf_cnpj_sem_mascara_prestador = None            
            
            
    telefone_str = None

    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', raw_texto_pil)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        telefone_prestatador_frame = telefone_str
    else:
        telefone_prestatador_frame = None 

             
    return cpf_cnpj_com_mascara_prestador, cpf_cnpj_sem_mascara_prestador, telefone_prestatador_frame, raw_texto_pil

In [ ]:
# Esta e o principio da melhor funcao do mundo
def extracao_pipeline(qualquer_df, fase, atividade, status):
    
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []
    imagens_list = []   
    
    pre_processo = ['nro_nota', 'competencia', 'dt_hr_emissao', 'codigo_verificacao', 'ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    time_now = cron.timenow_pt_BR()
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        seq_df = row['seq']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        pdf_pesquisavel = row['pdf_pesquisavel']
        one_page_doc = row['one_page']
        modelo = row['modelo']
        
        prefeitura = row['prefeitura']
        
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # 2. Busca modelo
        if (not status_documento == 'NAO_PROCESSAR') or (not document_type == 'outros') or (not one_page_doc == True):
            
            
            # 1. Analisa se PDF e pesquisavel
            if atividade_processo_atual == 'extracao_cabecalho':
                #print(f' 1 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel}\n')
                if status_documento == status_documento_atual:
                    if pdf_pesquisavel == True:
                        
                        nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section = extract_fields_cabecalho_pdf_pesquisavel(row, file_path)

                    elif pdf_pesquisavel == False:
                        
                        doc2convert = original_file_name
                       
                        image_2work, image_resized_name = convertResize(doc2convert, file_path, image_resized_path)
                        #image_2work, name_image_2work = convertResize(doc2convert, file_path, image_resized_path)
                        
                        #print(f' 0 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel} {name_image_2work}\n')
                        nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section, raw_texto = extract_fields_cabecalho_raster_pdf(row, file_path, image_2work)
                        
                        print(f'seq_df: {seq_df} | {original_file_name} | nro_nota: {nro_nota_frame} | competencia: {competencia_frame} | dt_hr_emissao: {dt_hr_emissao_frame} |  codigo_verificacao: {codigo_verificacao_frame}    raw_texto: raw_texto')
        
                    status_message = 'processado'
                    qualquer_df.loc[idx, 'date_time'] = time_now
                    qualquer_df.loc[idx, 'nro_nota'] = nro_nota_frame
                    qualquer_df.loc[idx, 'competencia'] = competencia_frame
                    qualquer_df.loc[idx, 'dt_hr_emissao'] = dt_hr_emissao_frame
                    qualquer_df.loc[idx, 'codigo_verificacao'] = codigo_verificacao_frame 
                    
            
            elif atividade_processo_atual == 'extracao_prestador':
            #print(f' 1 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel}\n')
                if status_documento == status_documento_atual:
                    if pdf_pesquisavel == True:
                        
                        nro_nota_frame, competencia_frame, dt_hr_emissao_frame, codigo_verificacao_frame, message_erro, section = extract_fields_cabecalho_pdf_pesquisavel(row, file_path)

                    elif pdf_pesquisavel == False:
                        
                        doc2convert = original_file_name
                        
                        image_2work, image_resized_name = convertResize(doc2convert, file_path, image_resized_path)
                        #image_2work, name_image_2work = convertResize(doc2convert, file_path, image_resized_path)
                        
                        #print(f' 0 - seq: {seq_df} | file: {original_file_name} |status_documento: {status_documento} pdf_pesquisavel: {pdf_pesquisavel} {name_image_2work}\n')
                        cpf_cnpj_com_mascara_prestador, cpf_cnpj_sem_mascara_prestador, telefone_prestatador_frame, raw_texto_pil = extract_fields_cnpj_prestador_raster_pdf(row, file_path, image_2work)
                        
                        print(f'seq_df: {seq_df} | file: {original_file_name} | {cpf_cnpj_com_mascara_prestador} ')
                        
                        inscricao_municipal_prestador_frame, inscricao_estadual_prestador_frame, msg, raw_texto_pil  = extract_fields_inscricao_prestador_raster_pdf(row, file_path, image_2work)
                        
                        print(f'seq_df: {seq_df} | file: {original_file_name} | {inscricao_municipal_prestador_frame} ')
                        
                        #cpf_cnpj_com_mascara_prestador, cpf_cnpj_sem_mascara_prestador, telefone_prestatador_frame, insc_muni_prest, insc_estad_prest, razao_prest, nome_fan_prest, end_prest, email_prest = extract_fields_prestador_raster_pdf(row, file_path, image_2work)
                        # 
                        
                        
        
                        # status_message = 'processado'
                        # qualquer_df.loc[idx, 'date_time'] = time_now
                        # qualquer_df.loc[idx, 'nro_nota'] = nro_nota_frame
                        # qualquer_df.loc[idx, 'competencia'] = competencia_frame
                        # qualquer_df.loc[idx, 'dt_hr_emissao'] = dt_hr_emissao_frame
                        # qualquer_df.loc[idx, 'codigo_verificacao'] = codigo_verificacao_frame         
                    
                

                                    


                i += 1
    total_map = i - 1
    print("Documentos mapeados: ", total_map)
    
    return raw_texto_pil
    


In [ ]:
# 3B. XXX Efetuo a extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'extracao_prestador' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'OK'

#raw_texto = extracao_pipeline(df_analise_pipe, fase, atividade, status)
raw_texto = extracao_pipeline(subset_df_analise_pipe, fase, atividade, status)

In [ ]:
raw_texto

In [ ]:
# XXX Recentemente criada para estrair dados Prestador - TEM QUE REFACTORIZAR
def bloco_prestador(row, file_path):
    
    message_erro = []        
    # 2. PRESTADOR DE SERVIÇO
    # Definir retângulo de interesse
    pdf_document = fitz.open(file_path)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]
    nf_data_prestador = {}
    x0 = 0
    y0 = 100
    x1 = 600
    y1 = 236  # Ajuste este valor para delimitar a região vertical

    # Extrair texto dentro do retângulo
    text = page.get_text("text", clip=(x0, y0, x1, y1))
    
    nf_data_prestador = Extc.extract_fields_prestador(text)
    try:
        razao_social = nf_data_prestador['razao_social']
        telefone = nf_data_prestador['telefone']
        cpf_cnpj_com_mascara = nf_data_prestador['cpf_cnpj_com_mascara']
        cpf_cnpj_sem_mascara = nf_data_prestador['cpf_cnpj_sem_mascara']
        inscricao_municipal = nf_data_prestador['inscricao_municipal']
        inscricao_estadual = nf_data_prestador['inscricao_estadual']
        nome_fantasia = nf_data_prestador['nome_fantasia']
        endereco = nf_data_prestador['endereco']
        email = nf_data_prestador['email']
    except Exception as e:
        msg = (f"doc: {original_file_name} |  {e}")
        message_erro.append(msg)
        razao_social = " "
        telefone = " "
        cpf_cnpj_com_mascara = " "
        cpf_cnpj_sem_mascara = " "
        inscricao_municipal = " "
        inscricao_estadual = " "
        razao_social = " "
        nome_fantasia = " "
        endereco = " "
        email = " "
    pdf_document.close()    
    return razao_social, telefone, cpf_cnpj_com_mascara, cpf_cnpj_sem_mascara, inscricao_municipal, inscricao_estadual, nome_fantasia, endereco, email, message_erro   

In [ ]:
# 3A. XXX Efetuo a extracao de dados
# analisar_pdf_pesquisavel
fase = 'analise' # 'pre-processamento'
atividade = 'extracao_cabecalho' #'pesquisar_prefeitura' # pesquisar_prefeitura  pesquisar_modelo    mensagem_status = "Reavaliar_PDF_Pesquisavel"
status = 'OK'

#raw_texto = extracao_pipeline(df_analise_pipe, fase, atividade, status)
raw_texto = extracao_pipeline(subset_df_analise_pipe, fase, atividade, status)

In [ ]:
raw_texto

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
subset_df_analise_pipe

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

In [ ]:
remove_images(imagens_list)

In [ ]:
df_analise_pipe

### Ajustando dados do DF e efetuando queries

In [ ]:
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)

In [ ]:
# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


In [ ]:
subset_df_analise_pipe.quert('nro_nota')

In [ ]:
# Efetuando pesquisa no DF
df_analise_pipe.query('pdf_pesquisavel == False')

In [ ]:
df_analise_pipe.query('nro_nota == 13')

In [ ]:
# XXX Para Analise de PDF Pesquisavel, uei uma copia do df_analise_pipe
subset_df_analise_pipe = df_analise_pipe

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq < 21 & seq > 10')

In [ ]:
subset_df_analise_pipe

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=13, column='model_frame', value=None)

<mark> <b>1.</b> Efetuando subset do df para pesquisa </mark>

In [ ]:
# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df_analise_pipe = df_analise_pipe.iloc[:10]

subset_df_analise_pipe

In [ ]:
# 1. Usando loc para Filtrar Baseado em Condições
subset_df = df_scan_pipe.loc[df_scan_pipe['directory'] == 'camaleao']

subset_df

In [ ]:
assert file_hash == file_hash2, "Os arquivos são diferentes"

In [ ]:
# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]

subset_df

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df = df_scan_pipe.query('directory == "root_dir" & seq < 10 & seq > 7')

subset_df

In [ ]:
# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

subset_df

In [ ]:
registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']

registro_específico

## Famigerado Outros

In [ ]:
                    acoes_necessarias = {}
                        
                    if nro_nota_frame == " ":
                        acoes_necessarias['nro_nota'] = True # 'nro_nota'

                    if competencia_frame == " ":
                        acoes_necessarias['competencia'] = True # 'competencia'

                    if dt_hr_emissao_frame == " ":
                        acoes_necessarias['dt_hr_emissao'] = True #'dt_hr_emissao'

                    if codigo_verificacao_frame == " ":
                        acoes_necessarias['codigo_verificacao'] = True #'codigo_verificacao'

                    info_list = []
                    seq_list = []
                    for action in pre_processo:
                        if action in acoes_necessarias:
                            seq = pre_processo.index(action) + 1  # Pega a sequência da ação baseada na lista pre_processo
                            n = seq - 1
                            seq_list.append(seq)
                            info = pre_processo[n]
                            info_list.append(info)
                            #print(f' 1 - seq_list: {seq_list} | info_list: {info_list} ')
                            
                            if (len(seq_list) > 4) and (1 not in seq_list):
                                status_documento = "REPROCESSAR"
                            elif (len(set(seq_list)) < len(seq_list)):
                                status_documento = "REVISAR - Erros duplicados"
                            elif (4 in seq_list) and (10 in seq_list):
                                status_documento = "REJEITAR - Erros críticos presentes"
                            else:
                                status_documento = "OK"
                                
                            # numero_erros = ','.join(seq_list) 
                            # descricao_erros = ','.join(info_list) 
                            
                            numero_erros = ','.join(map(str, seq_list))
                            descricao_erros = ','.join(info_list)

                            
                            qualquer_df.loc[idx, 's_act'] = numero_erros
                            qualquer_df.loc[idx, 'process_action'] = descricao_erros
                            qualquer_df.loc[idx, 'status_documento'] = status_documento

In [ ]:
                    # if razao_social == " ":
                    #     acoes_necessarias['razao_social'] = True #'razao_social'

                    # if cpf_cnpj_com_mascara == " ":
                    #     acoes_necessarias['cpf_cnpj_com_mascara'] = True #'cpf_cnpj_com_mascara'     

                    # if not pdf_realmente_pequisavel:
                    #     acoes_necessarias['ajustar_imagem'] =  True
                        
                    # if not doc_1_page:
                    #     acoes_necessarias['split_paginas'] = True
                    
                    # if new_name != original_file_name:
                    #     acoes_necessarias['ajustar_nome'] =  True
                        
                    # if not nome_prefeitura:
                    #     acoes_necessarias['buscar_nome_prefeitura'] = True    
                        
                    # if "cancelada" in original_file_name.lower():
                    #     acoes_necessarias['enviar_canceladas'] = True
                        
                    # if "listagem" in original_file_name.lower():
                    #     acoes_necessarias['enviar_listagens'] = True                   
                   
                   
                   #time_now = cron.timenow_pt_BR()
                    #bloco_1 = {
                    #     'document_unique_id': idx,
                    #     'seq': i,
                    #     'date_time': cron.timenow_pt_BR(),
                    #     'fase_processo': fase_atual,
                    #     'nome_atividade': nome_atividade,
                    #     'batch': batch_name,
                    #     'original_file_name': original_file_name,
                    #     'directory': file_dir,
                    #     'pdf_pesq': pdf_realmente_pequisavel,
                    #     'status_documento': status_documento, 
                    #     's_act': seq_list,
                    #     'process_action': info_list,
                    # }    
                    # bloco_1_list.append(bloco_1)
                    
                    # bloco_2 = { 
                    #     'document_unique_id': idx,   
                    #     'one_page': doc_1_page,
                    #     'prefeitura': nome_prefeitura,
                    #     'nro_nota': nro_nota,
                    #     'competencia':competencia,
                    #     'dt_hr_emissao':dt_hr_emissao,
                    #     'codigo_verificacao':codigo_verificacao,
                    #     'pages': nro_paginas,
                    #     'razao_social':razao_social,
                    #     'telefone': telefone,
                    # }    
                    # bloco_2_list.append(bloco_2)
                    
                    # bloco_3 = {
                    #     'document_unique_id': idx,    
                    #     'cpf_cnpj_com_mascara': cpf_cnpj_com_mascara,
                    #     'cpf_cnpj_sem_mascara': cpf_cnpj_sem_mascara,
                    #     'inscricao_municipal': inscricao_municipal,
                    #     'inscricao_estadual': inscricao_estadual,
                    #     'razao_social': razao_social,
                    #     'nome_fantasia': nome_fantasia,
                    #     'endereco': endereco,
                    #     'email': email,
                    #     'aditional_information': message_erro,
                    #     'file_hash': file_hash,
                    #     'file_path': file_path,
                    # }
                    # bloco_3_list.append(bloco_3) 

In [ ]:
import pandas as pd

# Criando um DataFrame de exemplo
data = {'A': [1, 2], 'B': [3, 4], 'Link': ['http://www.google.com', 'http://www.github.com']}
df = pd.DataFrame(data)

# Criando um escritor de Excel
with pd.ExcelWriter('path_to_file.xlsx', engine='xlsxwriter') as writer:
    
    # Escrevendo o DataFrame para o Excel
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    
    # Obtendo o objeto workbook e sheet
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Adicionando um formato de hyperlink
    hyperlink_format = workbook.add_format({'hyperlink': True, 'color': 'blue', 'underline': 1})
    
    # Fazendo um loop através dos links e aplicando o formato de hyperlink
    for row_num, link in enumerate(df['Link'], start=1):
        worksheet.write_url(row_num, df.shape[1] - 1, link, hyperlink_format)

# Agora, na coluna "Link" do seu arquivo Excel, você terá links clicáveis


In [ ]:
def extract_prefeitura(model, father, values):
    
    tipo = "sframe_field"
    data_extrated_prefeitura = {}
    #print(tipo)

    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

    for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
        
        label_value = row_sframe['label']
        
        #print("label_value", label_value)
        
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura

In [ ]:
        try:
            os.remove(file_delete_path)
        except Exception as e:
            print(f"Nao congui deletar: {e}") 
            
            df_analise_pipe, documentos = scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status)

In [ ]:
        try:
            os.remove(file_delete_path)
        except Exception as e:
            print(f"Nao congui deletar: {e}") 
            
            df_analise_pipe, documentos = scan_pipeline_documentos(documentos_scan_path, batch_name, fase, atividade, status)

In [ ]:
# Esta e o principio da melhor funcao do mundo
def analise_extracao_pipeline(qualquer_df, fase, atividade, status):
    
   
    linhas_analise = []
    bloco_1_list = []
    bloco_2_list = []
    bloco_3_list = []   
    
    pre_processo = ['ajustar_nome', 'split_paginas', 'ajustar_imagem', 'buscar_nome_prefeitura', 'enviar_canceladas', 'enviar_listagens']
    
    
    
    fase_processo_atual = fase
    atividade_processo_atual = atividade
    status_documento_atual = status
    
    i = 1
    for idx, row in qualquer_df.iterrows():
        message_erro = []
        
        # 1. Mapeamento de informacoes do DF
        document_unique_id = idx
        batch_name = row['batch']
        batch_name = row['batch']
        fase_processo = row['fase_processo']
        nome_atividade = row['nome_atividade']
        status_documento = row['status_documento']
        original_file_name = row['original_file_name']
        file_directory = row['directory']
        level = row['level']
        d_type = row['level']
        document_type = row['document_type']
        parent_document_unique_id = row['parent_document_unique_id']
        file_path = row['file_path']
        file_hash = row['file_hash']
        
        # Chama a função conv_filename
        #new_name = conv_filename(original_file_name)


        # 2. Busca modelo
        image_on_the_fly, image_resized_name = convertResizeAnalise_1page(original_file_name, file_path, image_resized_path)
        nome_prefeitura = pequisaModel(image_on_the_fly)
      
        # nro_nota, competencia, dt_hr_emissao, codigo_verificacao, message_erro = bloco_prefeitura(row, file_path)
        # razao_social, telefone, cpf_cnpj_com_mascara, cpf_cnpj_sem_mascara, inscricao_municipal, inscricao_estadual, nome_fantasia, endereco, email, message_erro = bloco_prestador(row, file_path)    

        # acoes_necessarias = {}
        
        # if not doc_1_page:
        #     acoes_necessarias['split_paginas'] = nro_paginas
        
        # if nro_nota == " ":
        #     acoes_necessarias['nro_nota'] = True # 'nro_nota'

        # if competencia == " ":
        #     acoes_necessarias['competencia'] = True # 'competencia'

        # if dt_hr_emissao == " ":
        #     acoes_necessarias['dt_hr_emissao'] = True #'dt_hr_emissao'

        # if codigo_verificacao == " ":
        #     acoes_necessarias['codigo_verificacao'] = True #'codigo_verificacao'

        # if razao_social == " ":
        #     acoes_necessarias['razao_social'] = True #'razao_social'

        # if cpf_cnpj_com_mascara == " ":
        #     acoes_necessarias['cpf_cnpj_com_mascara'] = True #'cpf_cnpj_com_mascara'     

        # if not pdf_realmente_pequisavel:
        #     acoes_necessarias['ajustar_imagem'] =  True
            
        # if not doc_1_page:
        #     acoes_necessarias['split_paginas'] = True
        
        # if new_name != original_file_name:
        #     acoes_necessarias['ajustar_nome'] =  True
            
        # if not nome_prefeitura:
        #     acoes_necessarias['buscar_nome_prefeitura'] = True    
            
        # if "cancelada" in original_file_name.lower():
        #     acoes_necessarias['enviar_canceladas'] = True
            
        # if "listagem" in original_file_name.lower():
        #     acoes_necessarias['enviar_listagens'] = True
                
        # info_list = []
        # seq_list = []
        # for action in pre_processo:
        #     if action in acoes_necessarias:
        #         seq = pre_processo.index(action) + 1  # Pega a sequência da ação baseada na lista pre_processo
        #         n = seq - 1
        #         seq_list.append(seq)
        #         info = pre_processo[n]
        #         info_list.append(info)
        #         if (len(seq_list) > 4) and (1 not in seq_list):
        #             status_documento = "REPROCESSAR"
        #         elif (len(set(seq_list)) < len(seq_list)):
        #             status_documento = "REVISAR - Erros duplicados"
        #         elif (4 in seq_list) and (10 in seq_list):
        #             status_documento = "REJEITAR - Erros críticos presentes"
        #         else:
        #             status_documento = "OK"  
                        
        #time_now = cron.timenow_pt_BR()
        # bloco_1 = {
        #     'document_unique_id': idx,
        #     'seq': i,
        #     'date_time': cron.timenow_pt_BR(),
        #     'fase_processo': fase_atual,
        #     'nome_atividade': nome_atividade,
        #     'batch': batch_name,
        #     'original_file_name': original_file_name,
        #     'directory': file_dir,
        #     'pdf_pesq': pdf_realmente_pequisavel,
        #     'status_documento': status_documento, 
        #     's_act': seq_list,
        #     'process_action': info_list,
        # }    
        # bloco_1_list.append(bloco_1)
        
        # bloco_2 = { 
        #     'document_unique_id': idx,   
        #     'one_page': doc_1_page,
        #     'prefeitura': nome_prefeitura,
        #     'nro_nota': nro_nota,
        #     'competencia':competencia,
        #     'dt_hr_emissao':dt_hr_emissao,
        #     'codigo_verificacao':codigo_verificacao,
        #     'pages': nro_paginas,
        #     'razao_social':razao_social,
        #     'telefone': telefone,
        # }    
        # bloco_2_list.append(bloco_2)
        
        # bloco_3 = {
        #     'document_unique_id': idx,    
        #     'cpf_cnpj_com_mascara': cpf_cnpj_com_mascara,
        #     'cpf_cnpj_sem_mascara': cpf_cnpj_sem_mascara,
        #     'inscricao_municipal': inscricao_municipal,
        #     'inscricao_estadual': inscricao_estadual,
        #     'razao_social': razao_social,
        #     'nome_fantasia': nome_fantasia,
        #     'endereco': endereco,
        #     'email': email,
        #     'aditional_information': message_erro,
        #     'file_hash': file_hash,
        #     'file_path': file_path,
        # }
        # bloco_3_list.append(bloco_3) 

        
        i += 1
    total_map = i - 1
    print("Documentos mapeados: ", total_map)
    
    # bloco_1_df = pd.DataFrame(bloco_1_list)
    # bloco_2_df = pd.DataFrame(bloco_2_list)
    # bloco_3_df = pd.DataFrame(bloco_3_list)
    
    # df_map_analise = pd.concat([bloco_1_df.set_index('document_unique_id'), 
    #                             bloco_2_df.set_index('document_unique_id'), 
    #                             bloco_3_df.set_index('document_unique_id')], axis=1)

    # df_map_analise = pd.DataFrame(linhas_analise)
    
    # return df_map_analise, bloco_1_list, bloco_2_list, bloco_3_list

In [ ]:
### IMPORTANTE - COPIA APENAS PARA REFERENCIA ######
#            NAO AJUSTAR SEM FALAR COMIGO          #
####################################################
# 7. XXX Pesquisa prefeitura no documento (dando as coordenadas) e efetuando o OCR
# 1. Geraçao do hash do arquivo
def generate_file_hash(file_path):
    # Abre o arquivo em modo de leitura de bytes
    with open(file_path, "rb") as f:
        # Lê o conteúdo do arquivo
        file_data = f.read()
        # Utiliza o algoritmo SHA-256 para gerar o hash
        file_hash = hashlib.sha256(file_data).hexdigest()
    return file_hash

# 2. Geraçao do Unique_id do arquivo
def generate_unique_id():
    return str(uuid.uuid4())

# 11. Pesquiso Unique_ID por file
def get_document_id_by_file(batch, file):
    
    result = filtrar_df(df_id_relations, Batch=batch, File=file)
    document_unique_id = result['Unique_ID'].values[0]
    
    return document_unique_id

def add_source_entry(batch_name, file_path, file, type, level, parent_unique_id):
    #unique_id = generate_unique_id(type)
    unique_id = generate_unique_id()
    time_now = cron.timenow_pt_BR()   
    file_hash = generate_file_hash(file_path) 
    if level == 1:
        parent_unique_id = unique_id
    data = {
        'Batch': batch_name,
        'Data': time_now,
        'File': file,
        'Type': type,
        'Level': level,
        'Unique_ID': unique_id,
        'Parent_Unique_ID': parent_unique_id,
        'Hash': file_hash,
        'File_Path': file_path
    }
    return data


In [ ]:
fase_processo = ['analise', 'pre-processamento', 'validacao', 'processamento', 'homologacao']
atividade = ['scan_analise', 'split-paginas', 'pre_process', 'validacao', 'processamento', 'homologacao']
status_documento = ['nao_processar', 'mapear', 'analisar', 'processar', 'avaliar', 'reprocessar', 'homologar']

In [ ]:
# IMPORTANTE - TRATAMENTO DE PATH

file_name = "Listagem de NFS_e _Sintético IM 237881_104871.pdf"

file_path2 = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_17/Listagem de NFS_e _Sintético IM 237881_104871.pdf"

diretorio_dir = os.path.relpath(file_path2, file_name)

diretorio = os.path.basename(file_path2)

sub_dir_path = os.path.normpath(file_path2)

sub_dir = os.path.join(root_output_dir, folder_name)

sub_dir_path = os.path.normpath(file_path2)

diretorio = os.path.relpath(file_path2, documentos_scan_path)

sub_dir = os.path.join(root_output_dir, folder_name)

In [ ]:
# Usando multiplos parametros (um dicionario)
df_id_relations = pd.DataFrame({
    'Batch': ['Batch_17', 'Batch_18'],
    'Unique_ID': [1, 2],
    'File': ['File1', 'File2'],
    'Type': ['Type1', 'Type2'],
    'Level': [1, 2],
})

In [ ]:
for idx, row in df_scan_pipe.iterrows():
    # Aqui, idx será o Unique_ID da linha atual
    atual_unique_id = idx
    original_file_name = row['original_file_name']
    print(f'processing: {idx} | file: {original_file_name:>75} | unique_id:{atual_unique_id:>30}')
    
    
    # ... (seu código de processamento)


df_scan_pipe.to_excel('df_scan_pipe3.xlsx', index=True)

In [ ]:
# 2. Estabelece o pipeline de documentos
def abertura_pipeline(index, batch_name, file, documentos_extracao_path, folder_name, file_path, content, parent_unique_id):
    
    time_now = cron.timenow_pt_BR()
    status_pro = "para_analise"
    new_unique_id = generate_unique_id(content)
    
    return {
        'index': index,
        'data_processamento': time_now,
        'Batch': batch_name,
        'diretorio_origem': folder_name,
        'nome_arquivo': file,
        'diretorio_origem': file_path,
        'status': status_pro,
        'Unique_ID': new_unique_id,
        'hash_value': generate_file_hash(file_path),
        'Parent_Unique_ID': parent_unique_id
    }


# 3. função para ajustar documentos no pipeline antes de serem processados
def pipeline_preprocessamento(documentos_extracao_path, content, unique_id):
    rows_list = []
    i = 0
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            try:
                i += 1
                new_row = abertura_pipeline(i, batch_name, file, documentos_extracao_path, folder_name, file_path, content, parent_unique_id)
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao criar nova row: {e}")
    
    df_documento_recebido = pd.DataFrame(rows_list)
    return df_documento_recebido    

In [ ]:
df_documento_recebido = pipeline_preprocessamento()

In [ ]:
unique_id = df_source[df_source['Batch'] == batch_name]['Unique_ID'].values[0]

In [ ]:
# 5. XXX Estabelece o pipeline de documentos
def abertura_pipeline(index, batch_name, file, documentos_extracao_path, folder_name, file_path):
    action_itens = {}
    
    status_pro = "para_analise"
    time_now = cron.timenow_pt_BR()
    
    
    return {
        'index': index,
        'data_processamento': time_now,
        'Batch': batch_name,
        'diretorio_origem': folder_name,
        'nome_arquivo': file,
        'diretorio_origem': file_path,
        'status': status_pro,
    }


# 3. funcao para ajustar documentos no pipeline antes de serem processados
def pipeline_preprocessamento():
    rows_list = []
    i = 0
    for root, dirs, files in os.walk(documentos_extracao_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            try:
                i += 1
                new_row = abertura_pipeline(i, batch_name, file, documentos_extracao_path, folder_name, file_path)
                rows_list.append(new_row)
            except Exception as e:
                print(f"Erro ao criar nova row: {e}") 
            
    
    df_documento_recebido = pd.DataFrame(rows_list)
    
    return df_documento_recebido